# Finding the nearest train/tube station

The purpose of this project is to create module that takes a London postcode input e.g. N4 4AF and will output the nearest train station to it.

In [1]:
import pandas as pd
import requests
from geopy import distance

In [2]:
# A csv file of london stations can be downloaded from this link
# https://www.doogal.co.uk/london_stations.php

# TODO join station data to tube line
gb_stations_df = pd.read_csv('gb_stations.csv')

In [3]:
GB_STATIONS = []
for x in gb_stations_df[['Station', 'Latitude', 'Longitude']].values:
    GB_STATIONS.append((x[0],(x[1],x[2])))

In [3]:
gb_stations_df.head()

,Station,Postcode,Latitude,Longitude,TLC,NLC,Owner,Entries and exits 2020,Entries and exits 2019,Entries and exits 2018,...,Interchanges 2014,Interchanges 2013,Interchanges 2012,Interchanges 2011,Interchanges 2010,Interchanges 2009,Interchanges 2008,Interchanges 2007,Interchanges 2006,Interchanges 2005
0,Abbey Wood,SE2 9RH,51.491061,0.121394,ABW,5131,TfL Rail,3825206,3769402.0,3124856.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aber,CF83 1AQ,51.574969,-3.229829,ABE,3813,TfW Rail,228480,245218.0,251108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abercynon,CF45 4SE,51.644712,-3.326989,ACY,3801,TfW Rail,282886,289008.0,298358.0,...,42347.0,35871.0,35304.0,33225.0,29324.0,28775.0,NaN,NaN,NaN,NaN
3,Aberdare,CF44 0PU,51.715062,-3.443084,ABA,3982,TfW Rail,504622,555780.0,571746.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aberdeen,AB11 6LX,57.143706,-2.098685,ABD,8976,ScotRail,2497108,2616142.0,2948186.0,...,219085.0,200864.0,197046.0,148594.0,164299.0,162526.0,119259.0,230788.0,201417.0,187983.0


In [5]:
# test_postcodes = ['N15 3AD', 'N4 4AF', 'N10 3QS', 'N15 4AR', 'N15 4JF']

In [12]:
# # TODO Could split this into two functions, one to retrieve an list of stations ranked in order or closeness
# # second one, just to grab the nearest
# # TODO round to 1 decimal place
# def find_nearest_london_station(postcode, stations_list):
#     pc_lat_lon = get_long_lat(postcode)['lat_lon']
#     distances_in_m = [(x[0],distance.distance(pc_lat_lon, x[1]).miles) for x in stations_list]
#     distances_in_m.sort(key=lambda x:x[1])
#     return(distances_in_m[0])

In [13]:
# find_nearest_london_station('*', gb_stations_tuples_list)

In [14]:
# find_nearest_london_station('*', gb_stations_tuples_list)

In [6]:
# TODO refactor - use less code
class NearestStation:
    
#     gb_stations = []
#     for x in gb_stations_df[['Station', 'Latitude', 'Longitude']].values:
#         gb_stations.append((x[0],(x[1],x[2])))
        
    def __init__(self, postcode):
        self.postcode = postcode
        self.endpoint = self._create_endpoint()
        self.output_dict = self._get_long_lat()
        self.pc_lat_lon = self.output_dict['lat_lon']
        self.nearest_station = self.find_nearest_station() #nearest_m TODO
        #TODO add nearest in km
    
    def _create_endpoint(self):
        pc = self.postcode.replace(' ','').lower()
        self.endpoint = f'http://api.postcodes.io/postcodes/{pc}'
        return self.endpoint

    def _get_long_lat(self):
        self.output_dict = {}
        r = requests.get(self.endpoint)
        self.output_dict['postcode'] = r.json()['result']['postcode']
        self.output_dict['lat_lon'] = (r.json()['result']['latitude'], r.json()['result']['longitude'])
        return self.output_dict
    
    def find_nearest_station(self): 
        #TODO round to 1 decimal place
        #TODO give user option of km or miles
        distances_in_m = [(x[0],distance.distance(self.pc_lat_lon, x[1]).miles) for x in GB_STATIONS]
        distances_in_m.sort(key=lambda x:x[1])
        return(distances_in_m[0])
    
    def find_nearest_n_stations(self,n):
        distances_in_m = [(x[0],distance.distance(self.pc_lat_lon, x[1]).miles) for x in GB_STATIONS]
        distances_in_m.sort(key=lambda x:x[1])
        return(distances_in_m[:n])
    

In [7]:
x = NearestStation('n4 4af')

In [8]:
x.nearest_station

('Harringay', 0.2894493888165714)

In [18]:
x.endpoint

'http://api.postcodes.io/postcodes/n44af'

In [19]:
x.find_nearest_n_stations(20)

[('Harringay', 0.2894493888165714),
 ('Harringay Green Lanes', 0.4287554239507669),
 ('Crouch Hill', 0.5076896853539483),
 ('Finsbury Park', 0.6148553411677796),
 ('Hornsey', 0.9557549938824251),
 ('Upper Holloway', 1.2181689691130957),
 ('Stamford Hill', 1.2557500321409345),
 ('Drayton Park', 1.4117167168997369),
 ('Seven Sisters', 1.4555995491303644),
 ('Stoke Newington', 1.5193234864059402),
 ('South Tottenham', 1.5324947553286579),
 ('Canonbury', 1.789054081159871),
 ('Alexandra Palace', 1.8200753425205685),
 ('Highbury and Islington', 1.8694302306857142),
 ('Rectory Road', 1.9080370252426713),
 ('Bruce Grove', 2.1098536269315904),
 ('Caledonian Road and Barnsbury', 2.1372053954643957),
 ('Dalston Kingsland', 2.162121228154882),
 ('Kentish Town', 2.164497975870116),
 ('Tottenham Hale', 2.2337101677745714)]

In [23]:
NearestStation('e17 3ss').nearest_station

('Wood Street', 0.3861582650348817)

In [24]:
# Clean station data?
NearestStation('e17 3ss').find_nearest_n_stations(5)

[('Wood Street', 0.3861582650348817),
 ('Walthamstow Central', 1.073987130513466),
 ('Highams Park', 1.141697589682547),
 ("Walthamstow Queen's Road", 1.2724699430085868),
 ('Leyton Midland Road', 1.5738249985800778)]